## Sliding window intensity assignment method, all (working) versions:
- i_sw 0.1 works, but leaves white edges, where the window doesn't reach
- i_sw 0.2 covers the edges!

In [ ]:
#run this for the codes to work :)

%pylab
%matplotlib inline
import numpy as np
from scipy import stats
import scipy.ndimage
import matplotlib.pyplot as plt
from images_segmentation import otsu as ot
from images_segmentation import dicescore as dsc

def crop(image, xmin, ymin, xmax, ymax):
    cropped=np.empty([xmax-xmin, ymax-ymin], dtype=float)
    for a,b in np.ndindex(xmax-xmin, ymax-ymin):
        cropped[a,b]=image[a+xmin,b+ymin]
    return cropped

def otsu_i(image, selectivity):
    img=ot.otsu_thresholding(image, 256)
    it = np.zeros([img.shape[0],img.shape[1],2])
    for x, y in np.ndindex(img.shape[0],img.shape[1]):
            if img[x,y] == 0:
                it[x,y,0]+=1
            else: 
                it[x,y,0]+=0
            it[x,y,1]+=selectivity
    return it

def edge(image, framesize):
    img = np.empty([image.shape[0]+framesize, image.shape[1]+framesize], dtype=int)
    for i,j in np.ndindex(image.shape[0],image.shape[1]):
        img[i,j]=image[i,j]
    figure()
    imshow(img,'gray')
    return img


## i_sw 0.1

In [ ]:
def i_sw(image,stepsize,framesize, sensitivity):
    img=copy(image)
    for i,j in np.ndindex(image.shape[0], image.shape[1]):
        img[i,j]=image[i,j]
    
    #create a 3 dimensional null array, the size of the image * 2, we can now save 2 separate values for each pixel
    array=np.empty([img.shape[0],img.shape[1],2], dtype=float)

    #set the first coordinates for iteration over the image
    x=0
    y=0

        #iterate crop function with a given size over the picture, while the "window" fits inside the image, 
        #for each window crop the image to window coordinates and perform otsu_i 
    
    while x+framesize<=img.shape[0]:
        while y+framesize<=img.shape[1]:
            window=crop(img,x,y,x+framesize, y+framesize)
            it = otsu_i(window, sensitivity)

            # for each pixel that fits into the current window, add the values from otsu_i 
            #generated array to according pixels in the big array
            for a, b in np.ndindex(window.shape[0], window.shape[1]):
                c=a+x
                d=b+y
                array[c,d,0]+=it[a,b,0]
                array[c,d,1]+=it[a,b,1]
            y+=stepsize
        y=0
        x+=stepsize

    # because array is 3 dimensional and picture is 2 dimensional the final intensities will
    # be determined by 'array', but saved into a final 'img' array (rewriting the copy of image)
    for i, j in np.ndindex(img.shape[0], img.shape[1]):
        if array[i,j,0]>array[i,j,1]:
            img[i,j]=0
        else:
            img[i,j]=255
    img=crop(img,0,0,image.shape[0],image.shape[1])
    figure()
    imshow(img,'gray')
    return img

## i_sw 0.2

In [ ]:
#method 1 : mode of foreground and background assignment
def i_sw(image,stepsize,framesize, sensitivity):

    #copy the image so it does not influence the original + add NA corners
    img=np.empty([image.shape[0]+framesize,image.shape[1]+framesize,])
    img[:]=np.NaN
    #img=copy(image)
    for i,j in np.ndindex(image.shape[0], image.shape[1]):
        img[i,j]=image[i,j]
    
    #create a 3 dimensional null array, the size of the image * 2, we can now save 2 separate values for each pixel
    array=np.empty([img.shape[0],img.shape[1],2], dtype=float)

    #set the first coordinates for iteration over the image
    x=0
    y=0

   

        #iterate crop function with a given size over the picture, while the "window" fits inside the image, 
        #for each window crop the image to window coordinates and perform otsu_i 
    
    while x+framesize<=img.shape[0]:
        while y+framesize<=img.shape[1]:
            window=crop(img,x,y,x+framesize, y+framesize)
            print(np.count_nonzero(np.isnan(window)))
            it = otsu_i(window, sensitivity)

            # for each pixel that fits into the current window, add the values from otsu_i 
            #generated array to according pixels in the big array
            for a, b in np.ndindex(window.shape[0], window.shape[1]):
                c=a+x
                d=b+y
                array[c,d,0]+=it[a,b,0]
                array[c,d,1]+=it[a,b,1]
            y+=stepsize
        y=0
        x+=stepsize

    # because array is 3 dimensional and picture is 2 dimensional the final intensities will
    # be determined by 'array', but saved into a final 'img' array (rewriting the copy of image)
    #img=crop(img,0,0,image.shape[0],image.shape[1])
    print(array)
    img=crop(img,0,0,image.shape[0],image.shape[1])
    for i, j in np.ndindex(img.shape[0], img.shape[1]):
        #img[i,j]=round(array[i,j,0]/(2*array[i,j,1]))
        if array[i,j,0]>array[i,j,1]:
            img[i,j]=0
        else:
            img[i,j]=255
    #img=crop(img,0,0,image.shape[0],image.shape[1])
    figure()
    imshow(img,'gray')
    return img

## i_sw 0.3
runtime reduction?

In [ ]:

def i_sw(image,stepsize,framesize, sensitivity):
    img=np.empty([image.shape[0]+framesize,image.shape[1]+framesize,])
    img[:]=np.NaN
    for i,j in np.ndindex(image.shape[0], image.shape[1]):
        img[i,j]=image[i,j]
    it=np.zeros([img.shape[0],img.shape[1],2])
    x=0
    y=0
    while x+framesize<=img.shape[0]:
        while y+framesize<=img.shape[1]:
            post_otsu = otsu_i(img[x:x+framesize,y:y+framesize], sensitivity)
            for a,b in np.ndindex(post_otsu.shape[0],post_otsu.shape[1]):
                it[x+a,y+b,0] +=post_otsu[a,b]
                it[x+a,y+b,1] += sensitivity
            y+=stepsize
        y=0
        x+=stepsize
    print(it)
    img=crop(img,0,0,image.shape[0],image.shape[1])
    for i, j in np.ndindex(img.shape[0], img.shape[1]):
        img[i,j]=round(it[i,j,0]/(2*it[i,j,1]))
    return img

In [ ]:
from PIL import Image
import os, os.path

imgs = []
path = r"data\Otsu_data\NIH3T3\img"
for f in os.listdir(path):
    imgs.append(imread(os.path.join(path,f)))

imgs_gt = []
path = r"data\Otsu_data\NIH3T3\gt"
for f in os.listdir(path):
    imgs_gt.append(imread(os.path.join(path,f)))

#imgs_1 = []
#path = r"data\Otsu_data\N2DH-GOWT1\img"
#for f in os.listdir(path):
   # imgs_1.append(imread(os.path.join(path,f)))

#imgs_gt_1 = []
#path = r"data\Otsu_data\N2DH-GOWT1\gt"
#for f in os.listdir(path):
   # imgs_gt_1.append(imread(os.path.join(path,f)))

#imgs_2 = []
#path = r"data\Otsu_data\N2DL-HeLa\img"
#for f in os.listdir(path):
    #imgs_2.append(imread(os.path.join(path,f)))

#imgs_gt_2 = []
#path = r"data\Otsu_data\N2DL-HeLa\gt"
#for f in os.listdir(path):
   # imgs_gt_2.append(imread(os.path.join(path,f)))

In [ ]:
isw_1 = []
for i in range(0,len(imgs)):
    isw_1.append(i_sw(imgs[i],50, 150, 0.7)) 

dsc_1 = []
for i in range(0,len(imgs)):
    dsc_1.append(min(dsc.f_dice(i,imgs_gt, isw_1), dsc.b_dice(i,imgs_gt,isw_1)))

print(dsc_1)
print(mean(dsc_1))

#isw_2 = []
#for i in range(0,len(imgs_1)):
    #otsu_2.append(ot.otsu_thresholding(imgs_1[i],220)) 

#isw_3 = []
#for i in range(0,len(imgs_2)):
    #otsu_3.append(ot.otsu_thresholding(imgs_2[i],3856)) 